In [168]:
import os
import json
import torch
import numpy as np
import pandas as pd

In [169]:
save_dir = '/home/kuba/Documents/Data/Datasets/Listerine'
raw_data_dir = '/home/kuba/Documents/Data/Raw/Listerine/3_final'

labels_to_one_hot = {
    'leftWater' : [[0],[0],[0],[1]],
    'leftLister' : [[0],[0],[1],[0]],
    'rightWater' : [[0],[1],[0],[0]],
    'rightLister' : [[1],[0],[0],[0]]

}

label_mapping = [
    ('left', 'water', 'leftWater'),
    ('left', 'listerine', 'leftLister'),
    ('right', 'water', 'rightWater'),
    ('right', 'listerine', 'rightLister')
]

In [170]:
def get_first_line(path):
    f = open(path)
    first_line = int(f.readline().strip().split(':')[1])
    return first_line

In [171]:
def separator(acc, gyro, labels):
    # Splits accelerometer and gyroscope data into segments based on activity labels
    # Returns lists of these data segments and their labels
    acc_bouts, gyro_bouts, y = [], [], []
    
    for side, liquid, label_key in label_mapping:
        for label in labels[side][liquid]:
            new_acc = [(acc.timestamp > label['start']) & (acc.timestamp < label['end'])]  
            acc_bouts.append(new_acc)
            
            new_gyro = [(gyro.timestamp > label['start']) & (gyro.timestamp < label['end'])]  
            gyro_bouts.append(new_gyro)
            
            y.append(labels_to_one_hot[label_key])
    
    return acc_bouts, gyro_bouts, y

In [172]:
def accumulator(dir):
    acc_full = []
    gyro_full = []
    y_full = []

    for recoding in sorted(os.listdir(dir)):
        full_path = os.path.join(dir,recoding)
        f = open(os.path.join(full_path,'labels.json'))
        labels = json.load(f)
        acc = pd.read_csv(os.path.join(full_path,'acceleration.csv'), skiprows=1)
        acc['timestamp']  = (acc['timestamp'] - acc['timestamp'].iloc[0]) * 1e-9 #subtract the start to get first time to be zero then convert from nano to sec
        #first_row_acc = get_first_line(os.path.join(full_path, 'acceleration.csv'))

        gyro = pd.read_csv(os.path.join(full_path,'gyroscope.csv'), skiprows=1)
        gyro['timestamp']  = (gyro['timestamp'] - gyro['timestamp'].iloc[0]) * 1e-9 #subtract the start to get first time to be zero then convert from nano to sec
        #first_row_gyro = get_first_line(os.path.join(full_path, 'gyroscope.csv'))
        # print(labels)


        acc, gyro, y = separator(acc, gyro, labels)
        acc_full.append(acc)
        gyro_full.append(gyro)
        y_full.append(y)
        
    print('here')
    return torch.tensor(acc_full), torch.tensor(gyro_full), torch.tensor(y_full)

In [173]:
for dir in sorted(os.listdir(raw_data_dir)):
    if dir != '.DS_Store':
        X_acc, X_gyro, y = accumulator(os.path.join(raw_data_dir,dir))
        print('hi')
        print(f'acc shape: {X_acc.shape} X gyro shape: {X_gyro.shape} y shpae: {y.shape}')
    #torch sve

here


ValueError: expected sequence of length 18 at dim 1 (got 17)